In [8]:
import json
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd

In [2]:
#load document
# file_path = 'data/doc2dial/doc2dial_qa_train.csv'
# loader = CSVLoader(file_path)
# document = loader.load()
# print(f'documents:{len(document)}')

In [9]:
#load qa pairs
file_path = 'data/doc2dial/doc2dial_qa_train.csv'
df = pd.read_csv(file_path)

In [6]:
for index, row in df.iterrows():
    print(index)
    if index == 10:
        break
    elif index % 5 == 0 and index != 0:
        break

0
1
2
3
4
5


In [3]:
import os
directory = 'data/doc2dial/failed_doc'
failed_doc_ids = [2,3,4]
if not os.path.exists(directory):
    os.makedirs(directory)

fail_filename = os.path.join(directory, 'failed_doc_ids.csv')

with open(fail_filename, 'w') as f:
    for item in failed_doc_ids:
        f.write("%s\n" % item)

In [6]:
string = '1. Forgetting to Update Address\nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be/npossible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse./nreceive their DRA assessment because we do not have their new address on record. Failure to pay the DRA will result in a suspended license. Learn more about About the NYS Driver Point System [4] and how to Pay Driver Responsibility Assessment [5 ]'

In [7]:
print(string)

1. Forgetting to Update Address
By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be/npossible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse./nreceive their DRA assessment because we do not have their new address on record. Failure to pay the DRA will result in a suspended license. Learn more about About the NYS Driver Point System [4] and how to Pay Driver Responsibility Assessment [5 ]


In [82]:
df['doc_id'][69]

'Top 5 DMV Mistakes and How to Avoid Them#3_0'

In [4]:
#iterate through qa_set

def iterator(df, test=True):
    cnt = 0
    for index, row in df.iterrows():
        print(row)
        cnt += 1
        if test and cnt == 1:
            break
        else:
            continue

iterator(df, test=True)

question      Hello, I forgot o update my address, can you h...
answer        hi, you have to report any change of address t...
domain                                                      dmv
doc_id             Top 5 DMV Mistakes and How to Avoid Them#3_0
references    [{'sp_id': '6', 'label': 'solution'}, {'sp_id'...
dial_id                        9f44c1539efe6f7e79b02eb1b413aa43
Name: 0, dtype: object


In [10]:
doc1= df.loc[0]
doc1

question      Hello, I forgot o update my address, can you h...
answer        hi, you have to report any change of address t...
domain                                                      dmv
doc_id             Top 5 DMV Mistakes and How to Avoid Them#3_0
references    [{'sp_id': '6', 'label': 'solution'}, {'sp_id'...
dial_id                        9f44c1539efe6f7e79b02eb1b413aa43
Name: 0, dtype: object

In [11]:
doc1['domain']

'dmv'

In [12]:
file_path = 'data/doc2dial/doc2dial_doc.json'
with open(file_path, 'r') as f:
    doc2dial_doc = json.load(f)

In [13]:
doc1_text = doc2dial_doc['doc_data'][doc1['domain']][doc1['doc_id']]['doc_text']
print(doc1_text)

Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them. 

1. Forgetting to Update Address 
By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be charged with operating an unregistered vehicle and/or aggravated unlicensed operation, both misdemeanors. This really happens , but the good news is this is a problem tha

# embedding

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.docstore.document import Document

In [15]:
def split(document):
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 0
    )
    # remove empty documents
    split_documents = text_splitter.split_documents(document)
    print(f'documents:{len(split_documents)}')
    return split_documents

def embedding(documents) -> FAISS:
    #load embeddings
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/gtr-t5-base')
    db = FAISS.from_documents(documents, embeddings)
    return db

def save_db(db):
    db.save_local("data/faiss_index")

def load_db(embeddings):
    new_db = FAISS.load_local("data/faiss_index", embeddings)
    return new_db

In [16]:
document = Document(page_content=doc1_text, metadata={"source": doc1['doc_id']})

In [17]:
split_documents = split([document])

documents:8


In [18]:
split_documents

[Document(page_content='Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them.', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'}),
 Document(page_content='1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be charged with operating an unregistered vehicle and/

In [19]:
db = embedding(split_documents)

/home/ygong/miniconda3/envs/atlas-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
save_db(db)

In [20]:
query = doc1['question']
docs = db.similarity_search(query)
docs

[Document(page_content='1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be charged with operating an unregistered vehicle and/or aggravated unlicensed operation, both misdemeanors. This really happens , but the good news is this is a problem that is easily avoidable. Learn more about how to change the address on your license and registrations [1 ]', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'}),
 Document(page_content='5. Not Bringing Proper Documentation to DMV Office \nAbout ten percent of 

In [53]:
len(docs[0].page_content)

804

In [58]:
tokenizer(docs[0].page_content, return_tensors="pt")['input_ids'].shape

torch.Size([1, 183])

In [18]:
doc2dial_doc['doc_data'][doc1['domain']][doc1['doc_id']]['spans']['56']

{'id_sp': '56',
 'tag': 'u',
 'start_sp': 4496,
 'end_sp': 4527,
 'text_sp': 'Sign up or log into MyDMV [6 ] ',
 'title': '5. Not Bringing Proper Documentation to DMV Office',
 'parent_titles': [],
 'id_sec': '15',
 'start_sec': 4496,
 'text_sec': 'Sign up or log into MyDMV [6 ] ',
 'end_sec': 4527}

# answer agent

In [21]:
from langchain import VectorDBQA
from langchain.chains import qa_with_sources
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
import os


In [23]:
#read from txt file
with open('api.txt', 'r') as f:
    lines = f.readlines()
    lines = [line.strip().split(":")[1] for line in lines]
    openai_api = lines[0]
    hf_api = lines[1]

In [24]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_api
os.environ["OPENAI_API_KEY"] = openai_api

In [25]:
qa_model = HuggingFaceHub(repo_id='google/flan-t5-large')
query = "Please answer the following question.\n"+doc1['question']
chain = load_qa_chain(llm=qa_model, chain_type="stuff")
model_answer = chain.run(input_documents=docs, question=query, raw_response=True)

In [26]:
docs

[Document(page_content='1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be charged with operating an unregistered vehicle and/or aggravated unlicensed operation, both misdemeanors. This really happens , but the good news is this is a problem that is easily avoidable. Learn more about how to change the address on your license and registrations [1 ]', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'}),
 Document(page_content='5. Not Bringing Proper Documentation to DMV Office \nAbout ten percent of 

In [28]:
model_answer

'b).'

In [47]:
import QAModel

In [48]:
flan_qa_model = AutoModelForSeq2SeqLM.from_pretrained("Intel/fid_flan_t5_base_nq")
qa_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [49]:
model_answer = QAModel.answer_from_local_model(doc1['question'], docs, tokenizer=qa_tokenizer, model=flan_qa_model)

Token indices sequence length is longer than the specified maximum sequence length for this model (765 > 512). Running this sequence through the model will result in indexing errors


In [40]:
model_answer

'DMV'

In [25]:
# from langchain.chains.qa_with_sources import load_qa_with_sources_chain

# chain = load_qa_with_sources_chain(qa_model, chain_type="map_reduce")
# chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [26]:
# from langchain import OpenAI

# query = "Please answer the following question using the given documents only.\n"+doc1['question']
# openai_model = OpenAI(model_name="text-davinci-003", max_tokens=1024)
# chain = load_qa_chain(llm=qa_model, chain_type="stuff")
# chain.run(input_documents=docs, question=query, raw_response=True)

# evaluation

In [27]:
import re

In [28]:
doc1['references']

"[{'sp_id': '6', 'label': 'solution'}, {'sp_id': '7', 'label': 'solution'}]"

In [29]:
df.loc[1]['references']

"[{'sp_id': '56', 'label': 'solution'}]"

In [30]:
res = re.findall(r"\d+", doc1['references'])
[int(i) for i in res]

[6, 7]

In [31]:
#global dictionary
# doc2dial_doc

def get_ref(doc):
    refs_ID = re.findall(r"\d+", doc['references'])
    refs_ID = [int(i) for i in refs_ID]
    sp_list = [doc2dial_doc['doc_data'][doc['domain']][doc['doc_id']]['spans'][str(i)] for i in refs_ID]
    return sp_list

get_ref(doc1)


[{'id_sp': '6',
  'tag': 'u',
  'start_sp': 346,
  'end_sp': 416,
  'text_sp': 'you must report a change of address to DMV within ten days of moving. ',
  'title': '1. Forgetting to Update Address',
  'parent_titles': [],
  'id_sec': '2',
  'start_sec': 333,
  'text_sec': 'By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ',
  'end_sec': 567},
 {'id_sp': '7',
  'tag': 'u',
  'start_sp': 416,
  'end_sp': 567,
  'text_sp': 'That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ',
  'title': '1. Forgetting to Update Address',
  'parent_titles': [],
  'id_sec': '2',
  'start_sec': 333,
  'text_sec': 'By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address

In [32]:
get_ref(doc1)[0]['title']

'1. Forgetting to Update Address'

In [33]:
get_ref(doc1)[0]['text_sp']

'you must report a change of address to DMV within ten days of moving. '

In [34]:
docs

[Document(page_content='1. Forgetting to Update Address', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'}),
 Document(page_content='By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'}),
 Document(page_content='possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'}),
 Document(page_content='receive their DRA assessment because we do 

In [35]:
docs[0].page_content

'1. Forgetting to Update Address'

In [36]:
em = 0
def eval_refs(refs, retrieved):
    #calculate exact match
    #calculate f1
    
    pass

In [37]:
doc2 = df.loc[1]
doc2

question                   Can I do my DMV transactions online?
answer        Yes, you can sign up for MyDMV for all the onl...
domain                                                      dmv
doc_id             Top 5 DMV Mistakes and How to Avoid Them#3_0
references               [{'sp_id': '56', 'label': 'solution'}]
dial_id                        9f44c1539efe6f7e79b02eb1b413aa43
Name: 1, dtype: object

In [38]:
short_answer = "By statute , you must report a change of address to DMV within ten days"

In [39]:
short_answer in doc1['answer']

False

In [62]:
example = {}
example['question'] = doc1['question']
example['answer'] = model_answer #TODO should be model result?
example['passage'] = docs[:5] #TODO only first document. If use all documents, need to change 

In [63]:
[document.page_content for document in example["passage"]]

['1. Forgetting to Update Address',
 'By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be',
 'possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.',
 'receive their DRA assessment because we do not have their new address on record. Failure to pay the DRA will result in a suspended license. Learn more about About the NYS Driver Point System [4] and how to Pay Driver Responsibility Assessment [5 ]']

In [64]:
"/n".join([document.page_content for document in example["passage"]])

'1. Forgetting to Update Address/nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be/npossible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse./nreceive their DRA assessment because we do not have their new address on record. Failure to pay the DRA will result in a suspended license. Learn more about About the NYS Driver Point System [4] and how to Pay Driver Responsibility Assessment [5 ]'

In [41]:
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer

PASSAGE_FORMAT = re.compile("« ([^»]*) » « ([^»]*) » (.*)")

# def format_passage_for_autoais(passage):
#   """Produce the NLI format for a passage.

#   Args:
#     passage: A passage from the Wikipedia scrape.

#   Returns:
#     a formatted string, e.g.

#       Luke Cage (season 2), Release. The second season of Luke Cage was released
#       on June 22, 2018, on the streaming service Netflix worldwide, in Ultra HD
#       4K and high dynamic range.
#   """
#   m = PASSAGE_FORMAT.match(passage)
#   if not m:
#     return passage

#   headings = m.group(2)
#   passage = m.group(3)
#   return f"{headings}. {passage}"

def format_example_for_autoais(example):
  return "premise: {} hypothesis: The answer to the question '{}' is '{}'".format(
      example["passage"], example["question"], example["answer"])

def infer_autoais(example, tokenizer, model):
  """Runs inference for assessing AIS between a premise and hypothesis.

  Args:
    example: Dict with the example data.
    tokenizer: A huggingface tokenizer object.
    model: A huggingface model object.

  Returns:
    A string representing the model prediction.
  """
  input_text = format_example_for_autoais(example)
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids
  outputs = model.generate(input_ids)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  inference = "Y" if result == "1" else "N"
  example["autoais"] = inference
  return inference

def score_predictions(predictions, nq_answers):
  """Scores model predictions against AutoAIS and NQ answers.

  Args:
    predictions: A dict from questions to prediction rows.
    nq_answers: A dict from questions to lists of NQ reference answers.
    passages: A dict from identifiers from the attribution corpus to the
      corresponding paragraphs.

  Returns:
    a dict of metric values, keyed by metric names
  """
  AUTOAIS = "google/t5_xxl_true_nli_mixture"
  hf_tokenizer = T5Tokenizer.from_pretrained(AUTOAIS)
  hf_model = T5ForConditionalGeneration.from_pretrained(AUTOAIS)

  autoais = 0
  target_answers = []
  predicted_answers = []
  for question, answers in nq_answers.items():
    target_answers.append(answers)
    example = predictions.get(question, None)
    if example is None:
    #   logging.error("Did not find prediction for '%s'", question)
      predicted_answers.append("")
      continue
    predicted_answers.append(example["answer"])
    if not example["passage"]:
      continue
    inference = infer_autoais(example, hf_tokenizer, hf_model)
    autoais += inference == "Y"

  scores = {}
  scores["AutoAIS"] = autoais / len(target_answers)
#   for metric, score in squad(target_answers, predicted_answers).items():
#     scores[f"SQuAD ({metric})"] = score
  return scores

In [42]:
# AUTOAIS = "google/t5_xxl_true_nli_mixture"
# hf_tokenizer = T5Tokenizer.from_pretrained(AUTOAIS)
# hf_model = T5ForConditionalGeneration.from_pretrained(AUTOAIS)

In [43]:
# infer_autoais(example, hf_tokenizer, hf_model)

In [44]:
example

{'question': 'Hello, I forgot o update my address, can you help me with that?',
 'answer': 'a suspended license',
 'passage': Document(page_content='1. Forgetting to Update Address', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'})}

In [45]:
[term['text_sp'] for term in get_ref(doc1)]

['you must report a change of address to DMV within ten days of moving. ',
 'That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ']

In [46]:
docs

[Document(page_content='1. Forgetting to Update Address', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'}),
 Document(page_content='By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'}),
 Document(page_content='possible mail correspondence can reach you. Also , turning in your plates is important to avoid an insurance lapse.', metadata={'source': 'Top 5 DMV Mistakes and How to Avoid Them#3_0'}),
 Document(page_content='receive their DRA assessment because we do 

In [47]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_f1(model_answer, truth):
    pred_tokens = normalize_text(model_answer).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [48]:
model_answer

'a suspended license'

In [49]:
docs[0].page_content

'1. Forgetting to Update Address'

In [50]:
compute_f1(model_answer, docs[0].page_content)

0

In [67]:
df.loc[0]['question'] == example['question']

True

In [54]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")